In [ ]:
import numpy as np
import cupy as cp
import pickle
from keras.datasets import mnist

def load_mnist_dataset():
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    train_images = cp.array(train_images.reshape((train_images.shape[0], 28, 28, 1))).astype('float32') / 255
    test_images = cp.array(test_images.reshape((test_images.shape[0], 28, 28, 1))).astype('float32') / 255
    return train_images, train_labels, test_images, test_labels

def print_progress_bar(iteration, total, prefix='', suffix='', decimals=1, length=50, fill='█'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end='\r')
    # Print New Line on Complete
    if iteration == total:
        print()

train_images, train_labels, test_images, test_labels = load_mnist_dataset()

class cnn:
    def __init__(self, input_shape, num_filters, filter_size, pool_size, output_size):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.filter = cp.random.randn(num_filters, filter_size[0], filter_size[1], input_shape[2]) * 0.01
        self.pool_size = pool_size

        conv_output_height = (input_shape[0] - filter_size[0] + 1) // pool_size[0]
        conv_output_width = (input_shape[1] - filter_size[1] + 1) // pool_size[1]

        if conv_output_height <= 0 or conv_output_width <= 0:
            raise ValueError("Invalid dimensions for convolution and pooling layers.")

        flattened_size = conv_output_height * conv_output_width * num_filters
        self.weights_fc = cp.random.randn(flattened_size, output_size) * 0.01
        self.bias_fc = cp.zeros((1, output_size))

    def sigmoid(self, z):
        return 1 / (1 + cp.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def convolve(self, input_images, filter):
        batch_size, image_height, image_width, _ = input_images.shape
        filter_height, filter_width, _ = filter.shape[1:4]
        output_height = image_height - filter_height + 1
        output_width = image_width - filter_width + 1

        if output_height <= 0 or output_width <= 0:
            raise ValueError("Invalid dimensions for convolution.")

        result = cp.zeros((batch_size, output_height, output_width, self.num_filters))

        for f in range(self.num_filters):
            for b in range(batch_size):
                for i in range(output_height):
                    for j in range(output_width):
                        result[b, i, j, f] = cp.sum(
                            input_images[b, i:i+filter_height, j:j+filter_width, :] * filter[f]
                        )

        return result

    def pool(self, x):
        batch_size, height, width, num_filters = x.shape
        pooled_height = height // self.pool_size[0]
        pooled_width = width // self.pool_size[1]

        result = cp.zeros((batch_size, pooled_height, pooled_width, num_filters))

        for b in range(batch_size):
            for f in range(num_filters):
                for i in range(pooled_height):
                    for j in range(pooled_width):
                        result[b, i, j, f] = cp.max(
                            x[b, i*self.pool_size[0]:(i+1)*self.pool_size[0],
                                j*self.pool_size[1]:(j+1)*self.pool_size[1], f]
                        )

        return result

    def flatten(self, x):
        return x.reshape(x.shape[0], -1)

    def forward(self, img):
        if len(img.shape) == 3:
            img = img[cp.newaxis, ...]

        if img.shape[1:] != self.input_shape:
            raise ValueError(f"Input shape must be {self.input_shape}, but got {img.shape[1:]}")

        conv_output = self.convolve(img, self.filter)
        pooled_output = self.pool(conv_output)
        flattened_output = self.flatten(pooled_output)
        z_fc = cp.dot(flattened_output, self.weights_fc) + self.bias_fc
        a_fc = self.sigmoid(z_fc)
        return a_fc

    def backward(self, x, y, output, learning_rate):
        error = output - y
        d_weights_fc = cp.dot(self.flatten(self.pool(self.convolve(x, self.filter))).T, error)
        self.weights_fc -= learning_rate * d_weights_fc
        self.bias_fc -= learning_rate * error

    def train(self, x, y, epochs, learning_rate):
        for epoch in range(epochs):
            for i in range(len(x)):
                img = x[i].reshape((1,) + self.input_shape)
                output = self.forward(img)
                self.backward(img, y[i], output, learning_rate)

    def predict(self, x):
        if len(x.shape) != 3 or x.shape[1:] != self.input_shape:
            raise ValueError(f"Input shape must be {self.input_shape}, but got {x.shape}")
        output = self.forward(x[cp.newaxis, ...])
        return cp.argmax(output, axis=1)

    def save_weights(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump([cp.asnumpy(self.filter), cp.asnumpy(self.weights_fc), cp.asnumpy(self.bias_fc)], f)

    def load_weights(self, filename):
        with open(filename, 'rb') as f:
            self.filter, self.weights_fc, self.bias_fc = [cp.array(arr) for arr in pickle.load(f)]

num_labels = 10
learning_rate = 0.1
epochs = 1
train_labels_one_hot = np.eye(num_labels)[train_labels]

train_images = cp.array(train_images)
train_labels_one_hot = cp.array(train_labels_one_hot)
test_images = cp.array(test_images)
test_labels = cp.array(test_labels)

num_filters = 3
filter_size = (3, 3)
pool_size = (2, 2)
input_shape = (28, 28, 1)
output_size = 10

network = cnn(input_shape, num_filters, filter_size, pool_size, output_size)

total_batches = len(train_images)
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    for i in range(0, len(train_images)):
        img = train_images[i].reshape((1,) + network.input_shape)
        label = train_labels_one_hot[i].reshape((1, -1))
        network.train(img, label, 1, learning_rate)
        print_progress_bar(i + 1, total_batches, prefix='Progress:', suffix='Complete', length=50)

# Testing the network
predictions = cp.array([network.predict(img) for img in test_images])
accuracy = cp.mean(predictions == test_labels)
print('Test Accuracy:', accuracy)

Epoch 1/1


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import random

# Button for saving weights
save_button = widgets.Button(description="Save Weights")
def save_button_clicked(b):
    # Specify the filename
    network.save_weights("network_weights.pkl")
    print("Weights saved successfully.")
save_button.on_click(save_button_clicked)

# Button for loading weights
load_button = widgets.Button(description="Load Weights")
def load_button_clicked(b):
    # Specify the filename
    network.load_weights("network_weights.pkl")
    print("Weights loaded successfully.")
load_button.on_click(load_button_clicked)

# Button for running the model
run_button = widgets.Button(description="Run Model")
def run_button_clicked(b):
    clear_output(wait=True)
    indices = random.sample(range(len(test_images)), 10)
    for i in range(0, 9):
        image = test_images[i].reshape(network.input_shape)
        plt.imshow(image, cmap='gray')
        plt.show()
        output = network.forward(image)
        predicted_label = np.argmax(output, axis=1)
        print("Predicted Label:", predicted_label[0])
run_button.on_click(run_button_clicked)

# Display the buttons
display(save_button, load_button, run_button)